In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from IPython.display import Markdown
import re
import datetime
from pykrx.stock import get_market_ticker_list, get_market_ticker_name
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.tools import tool
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from markitdown import MarkItDown
import dotenv
import os

dotenv.load_dotenv()

True

In [3]:
import yfinance as yf

# 피엔티 , 137400

In [ ]:
class State(TypedDict):
    messages : Annotated[list, add_messages]
    df : Annotated[pd.DataFrame, "Stock Dataset"]
    tool_call : Annotated[str, "tool_call Result"]

In [72]:
s = State["df"]

In [5]:
# 주식 DB를 만드는 함수 {"삼성전자":"005930, ... }

def create_stock_db():
    stock_dict = {}
    today = datetime.datetime.today()
    
    stock_list = get_market_ticker_list(today, market="KOSPI")

    for stock in stock_list:
        stock_dict.update({get_market_ticker_name(stock):stock})
    
    stock_list = get_market_ticker_list(today, market="KOSDAQ")

    for stock in stock_list:
        stock_dict.update({get_market_ticker_name(stock):stock})

    return stock_dict

In [6]:
stock_db = create_stock_db()

In [7]:
prompt = PromptTemplate.from_template("""
당신은 주식 이름 추출기입니다.
주어진 문장에서 주식이름만 추출하세요.

### 예시 1
query : 삼성전자의 최근 1년에 대해서 분석해주세요.

answer : 삼성전자

### 예시 2
query : AJ홀딩스우의 최근 실적은 얼마인가요?

answer : AJ홀딩스우

### 입력
query : {query}

answer : 

"""
)

llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0,)

In [8]:
stock_name = "피엔티"

stock_code = stock_db[stock_name]


In [9]:
@tool
def search_stock(query):

    """
    주식 검색 도구입니다.
    결과값으로 데이터프레임이 반환됩니다.
    입력 쿼리에서 주식이름을 추출한 후 모든 주식 데이터를 가져옵니다.
    """


    prompt = PromptTemplate.from_template("""
        당신은 주식 이름 추출기입니다.
        주어진 문장에서 주식이름만 추출하세요.

        ### 예시 1
        query : 삼성전자의 최근 1년에 대해서 분석해주세요.

        answer : 삼성전자

        ### 예시 2
        query : AJ홀딩스우의 최근 실적은 얼마인가요?

        answer : AJ홀딩스우

        ### 입력
        query : {query}

        answer : 

        """
        )
    
    chain = prompt | llm | StrOutputParser()

    stock_name = chain.invoke({"query":query})

    try:
        stock_code = stock_db[stock_name.strip().upper()]   
    except:
        raise ValueError(f"종목명 : {stock_name}을/를 검색할 수 없습니다. 오탈자나 한국거래소에서 거래중인 주식인지 확인해주세요.")
    
    # 예: 삼성전자 (한국거래소는 뒤에 '.KS'를 붙임)
    ticker = yf.Ticker(stock_code+".KS")

    df = ticker.history(period="max") # 기간: '1d', '5d', '1mo', '1y', 'max' 등

    return df

In [10]:
df = search_stock("LS가 뭐죠?")

C:\Users\Admin\AppData\Local\Temp\ipykernel_8944\546577314.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  df = search_stock("LS가 뭐죠?")
keys must be str, int, float, bool or None, not Timestamp
Error in LangChainTracer.on_tool_end callback: TypeError('keys must be str, int, float, bool or None, not Timestamp')


In [37]:
search_stock.invoke("LS가 뭐죠?")

Error in LangChainTracer.on_tool_end callback: TypeError('keys must be str, int, float, bool or None, not Timestamp')


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-04 00:00:00+09:00,8647.702567,10137.029120,8647.702567,10088.986328,509500,600.0,0.0
2000-01-05 00:00:00+09:00,10088.986328,10785.606813,9656.601200,10088.986328,457500,0.0,0.0
2000-01-06 00:00:00+09:00,10329.202588,10329.202588,9128.132520,9608.560547,351700,0.0,0.0
2000-01-07 00:00:00+09:00,9224.217965,9704.645984,9224.217965,9560.517578,191100,0.0,0.0
2000-01-10 00:00:00+09:00,9608.560380,9752.688786,9272.260767,9560.517578,198600,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-17 00:00:00+09:00,130600.000000,131300.000000,121200.000000,121700.000000,725086,0.0,0.0
2025-03-18 00:00:00+09:00,122400.000000,127100.000000,121600.000000,122800.000000,554183,0.0,0.0
2025-03-19 00:00:00+09:00,121200.000000,122700.000000,117800.000000,118000.000000,420890,0.0,0.0


In [63]:
df_dict = df.reset_index().to_dict()

In [64]:
pd.DataFrame(df_dict)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2000-01-04 00:00:00+09:00,8647.703404,10137.030101,8647.703404,10088.987305,509500,600.0,0.0
1,2000-01-05 00:00:00+09:00,10088.987305,10785.607857,9656.602134,10088.987305,457500,0.0,0.0
2,2000-01-06 00:00:00+09:00,10329.199438,10329.199438,9128.129736,9608.557617,351700,0.0,0.0
3,2000-01-07 00:00:00+09:00,9224.213254,9704.641027,9224.213254,9560.512695,191100,0.0,0.0
4,2000-01-10 00:00:00+09:00,9608.555473,9752.683805,9272.256031,9560.512695,198600,0.0,0.0
...,...,...,...,...,...,...,...,...
6308,2025-03-17 00:00:00+09:00,130600.000000,131300.000000,121200.000000,121700.000000,725086,0.0,0.0
6309,2025-03-18 00:00:00+09:00,122400.000000,127100.000000,121600.000000,122800.000000,554183,0.0,0.0
6310,2025-03-19 00:00:00+09:00,121200.000000,122700.000000,117800.000000,118000.000000,420890,0.0,0.0
6311,2025-03-20 00:00:00+09:00,119500.000000,125700.000000,118300.000000,123300.000000,473135,0.0,0.0


In [ ]:
df_json = df.reset_index().to_json(,)

In [54]:
df_json["Date"]

TypeError: string indices must be integers

In [11]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-04 00:00:00+09:00,8647.703404,10137.030101,8647.703404,10088.987305,509500,600.0,0.0
2000-01-05 00:00:00+09:00,10088.987305,10785.607857,9656.602134,10088.987305,457500,0.0,0.0
2000-01-06 00:00:00+09:00,10329.199438,10329.199438,9128.129736,9608.557617,351700,0.0,0.0
2000-01-07 00:00:00+09:00,9224.213254,9704.641027,9224.213254,9560.512695,191100,0.0,0.0
2000-01-10 00:00:00+09:00,9608.555473,9752.683805,9272.256031,9560.512695,198600,0.0,0.0


In [162]:
@tool
def web_search(query):
    """
    입력된 질의를 바탕으로 검색을 시도하는 도구입니다.
    기본적으로 5개의 검색 결과를 가져옵니다.
    """
    tool = TavilySearchResults()
    return tool

In [ ]:
tools = [PythonAstREPLTool(), TavilySearchResults(), search_stock]

In [33]:
code_tool = PythonAstREPLTool(name="code_tool")

In [13]:
code_tool.invoke("1+1")

2

In [34]:
with_tool = llm.bind_tools([code_tool])

In [18]:
result = with_tool.invoke("한국의 평균기온은 어떻게 되나요?")

In [22]:
result

AIMessage(content='한국의 평균 기온은 지역과 계절에 따라 다르지만, 일반적으로 다음과 같은 경향을 보입니다:\n\n1. **겨울 (12월~2월)**: 평균 기온은 -6도에서 3도 사이입니다. 북부 지역은 더 춥고, 남부 지역은 상대적으로 온화합니다.\n2. **봄 (3월~5월)**: 평균 기온은 5도에서 20도 사이로, 따뜻한 날씨가 시작됩니다.\n3. **여름 (6월~8월)**: 평균 기온은 20도에서 30도 사이로, 특히 7월과 8월은 매우 덥고 습합니다.\n4. **가을 (9월~11월)**: 평균 기온은 10도에서 20도 사이로, 기온이 서서히 내려갑니다.\n\n전국 평균 기온은 대략 10도에서 15도 사이로 볼 수 있습니다. 하지만 이는 대략적인 수치이며, 기후 변화나 특정 연도의 날씨에 따라 달라질 수 있습니다. 더 구체적인 데이터가 필요하다면 기상청의 자료를 참고하는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 260, 'prompt_tokens': 98, 'total_tokens': 358, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e4fa3702df', 'finish_reason': 'stop', 'logprobs': None}, id='run-6cc5f3d3-26f3-4b57-8b45-d72f7fc38f11-0', usage_metadata={'inpu

In [35]:
result = with_tool.invoke("1~10까지 반복하는 파이썬 코드를 짜고싶어")

In [36]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bRtYkrFQbPI8xYmXKD23JJw2', 'function': {'arguments': '{"query":"for i in range(1, 11):\\n    print(i)"}', 'name': 'code_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 103, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e4fa3702df', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dec3caa3-2141-40d9-b7dc-dd853d15ceee-0', tool_calls=[{'name': 'code_tool', 'args': {'query': 'for i in range(1, 11):\n    print(i)'}, 'id': 'call_bRtYkrFQbPI8xYmXKD23JJw2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 103, 'output_tokens': 30, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache

In [166]:
llm_with_tools = llm.bind_tools(tools)

In [173]:
result = llm_with_tools.stream("피엔티의 최근 실적은 어떤가요?", stream_mode="values")

In [172]:
for step in result:
    print(step)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_eJlueOyRz6qlY1dDtoBjtKwa', 'function': {'arguments': '', 'name': 'search_stock'}, 'type': 'function'}]} response_metadata={} id='run-c19068de-2ea6-4e2c-addb-d100c1dbbffe' tool_calls=[{'name': 'search_stock', 'args': {}, 'id': 'call_eJlueOyRz6qlY1dDtoBjtKwa', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'search_stock', 'args': '', 'id': 'call_eJlueOyRz6qlY1dDtoBjtKwa', 'index': 0, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]} response_metadata={} id='run-c19068de-2ea6-4e2c-addb-d100c1dbbffe' tool_calls=[{'name': '', 'args': {}, 'id': None, 'type': 'tool_call'}] tool_call_chunks=[{'name': None, 'args': '{"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'query', 'name': None}, 'type': None}]} response_

In [175]:
for step in result:
    step.pretty_print()

In [164]:
ToolNode(tools)

tools(tags=None, recurse=True, explode_args=False, func_accepts_config=True, func_accepts={'store': ('__pregel_store', None)}, tools_by_name={'python_repl_ast': PythonAstREPLTool(globals={}, locals={}), 'tavily_search_results_json': TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))), 'search_stock': StructuredTool(name='search_stock', description='주식 검색 도구입니다.\n결과값으로 데이터프레임이 반환됩니다.\n입력 쿼리에서 주식이름을 추출한 후 모든 주식 데이터를 가져옵니다.', args_schema=<class 'langchain_core.utils.pydantic.search_stock'>, func=<function search_stock at 0x0000028250B3E950>)}, tool_to_state_args={'python_repl_ast': {}, 'tavily_search_results_json': {}, 'search_stock': {}}, tool_to_store_arg={'python_repl_ast': None, 'tavily_search_results_json': None, 'search_stock': None}, handle_tool_errors=True, messages_key='messages')

In [157]:
web_search = TavilySearchResults()

In [161]:
web_search.invoke("피엔티는 어떤 기업인가요?")

[{'title': "[기업분석] 배터리랑 같이 크는 장비 업체 '피엔티' - 바이라인네트워크",
  'url': 'https://byline.network/2022/07/15-195/',
  'content': '피엔티는 이차전지를 구성하는 전극과 분리막, 전자소재 등을 제조하는 장비와 자동화 설비를 제공하는 국내 업체입니다. 2003년에 설립했고 2013년에',
  'score': 0.8282873},
 {'title': '회사소개 | (주)피엔티',
  'url': 'https://www.epnt.co.kr/company/',
  'content': '주식회사 피앤티는 철저한 장인정신과 미래를 개척하는 도전정신으로 감동을 주는 회사, 신뢰를 주는 롤투롤 설비 생산 기업 입니다.',
  'score': 0.7001659555555555},
 {'title': '보유종목점검 - 1. 피엔티 : 네이버 블로그',
  'url': 'https://blog.naver.com/chlbest/223714149861',
  'content': '피엔티, 현대차 의왕 연구소에 전고체 전극 장비 공급\u200b 피엔티 공장 조감도(사진=회사 홈페이지)이차전지 제조 장비 전문 기업인 피엔티(PNT)가 현대자동',
  'score': 0.57435945},
 {'title': '(주)피엔티 2025년 기업정보 | 직원수, 근무환경, 복리후생 등 - 사람인',
  'url': 'https://www.saramin.co.kr/zf_user/company-info/view/csn/Zm9qNG9WcGt6dlp0RWxvelMrSnVHUT09/company_nm/(%EC%A3%BC)%ED%94%BC%EC%97%94%ED%8B%B0',
  'content': '피엔티는 각종 롤투롤 컨버팅 기계를 주력으로 성장한 회사입니다. 웹가이딩, 장력제어, 오토터렛, 오토스플라이싱, 코팅, 칼렌더링, 슬리팅 등 기반이 되는 기술에 강점',
  'score': 0.48574239999999996},
 

In [74]:
tool_calls = []

In [ ]:
def route_tools(
    state: State,
):
try:
        message = messages[-1]
    else:
        # 입력 상태에 메시지가 없는 경우 예외 발생
        raise ValueError(f"No messages found in input state to tool_edge: {state}")

    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    if hasattr(message, "tool_calls") and len(message.tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tools"
    # 도구 호출이 없는 경우 "END" 반환
    return "llm"